In [6]:
!python --version

Python 3.10.12


In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBRegressor
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import joblib

In [5]:
sklearn.__version__

'1.2.2'

In [3]:
joblib.__version__

'1.4.2'

In [5]:
def drop_columns(df, columns_to_drop):
    """
    Drop specified columns from a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame from which columns should be dropped.
    columns_to_drop (list): A list of column names to be dropped.

    Returns:
    pd.DataFrame: The DataFrame with specified columns dropped.
    """
    # Drop the specified columns
    df_dropped = df.drop(columns=columns_to_drop)
    return df_dropped

## Prep for pileine

In [6]:
data = pd.read_csv('/content/FE_RoomBasePrice.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 49 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   unit_type_name                 142 non-null    object 
 1   property_design                142 non-null    float64
 2   property_type                  142 non-null    float64
 3   number_of_bookings             142 non-null    int64  
 4   bedroom                        142 non-null    float64
 5   bathroom                       142 non-null    float64
 6   beds                           142 non-null    float64
 7   capacity                       142 non-null    float64
 8   ac                             142 non-null    float64
 9   balcony                        142 non-null    float64
 10  beachfront                     142 non-null    float64
 11  breakfast                      142 non-null    float64
 12  building_staff                 142 non-null    flo

In [7]:
columnsToDropBeforeTraining = ['Unnamed: 0.1','Unnamed: 0', 'room_id']
data = drop_columns(data, columnsToDropBeforeTraining)

In [8]:
average_baseline_price = data['average_baseline_price']
data = data.drop(columns='average_baseline_price')

In [9]:
X = data
y = average_baseline_price

In [25]:
# Example column names
numerical_features = ['bedroom','bathroom','beds','capacity','lat','lng','distance_to_coastline','area_distance_to_airport','total_fas','ratio_bedroom_bathroom','ratio_bedroom_cap',
                  'avg_price_distance_to_coast', 'avg_price_distance_to_airport', 'avg_price_bedroom','avg_price_beds','avg_price_bathroom','avg_price_total_fas']
categorical_features = ['unit_type_name', 'property_design', 'property_type','area_name']
unprocessed_features = ['ac', 'balcony', 'beachfront', 'breakfast', 'building_staff',
                             'cable_tv', 'essentials', 'garden', 'gym', 'hair_dryer',
                             'hanger', 'heating', 'hot_water', 'kitchen', 'linens',
                             'lock', 'luggage_drop_off', 'parking', 'pool',
                             'private_entrance', 'shampoo', 'tv', 'washer', 'wifi',
                             'workspace']

# Get indices of these columns
numerical_indices = [data.columns.get_loc(col) for col in numerical_features]
categorical_indices = [data.columns.get_loc(col) for col in categorical_features]
unprocessed_indices = [data.columns.get_loc(col) for col in unprocessed_features]

In [32]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_indices),
        ('cat', OneHotEncoder(), categorical_indices),
        ('passthrough', 'passthrough', unprocessed_indices)
    ]
)

In [27]:
TEST_RATIO = 0.3
RANDOM_STATE = 123
params = {
    "n_estimators": 1000,
    "max_depth": 64,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "absolute_error",
}

In [28]:
model = RandomForestRegressor(n_estimators=58,max_depth=params["max_depth"],random_state=RANDOM_STATE)

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 48 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   unit_type_name                 142 non-null    object 
 1   property_design                142 non-null    float64
 2   property_type                  142 non-null    float64
 3   number_of_bookings             142 non-null    int64  
 4   bedroom                        142 non-null    float64
 5   bathroom                       142 non-null    float64
 6   beds                           142 non-null    float64
 7   capacity                       142 non-null    float64
 8   ac                             142 non-null    float64
 9   balcony                        142 non-null    float64
 10  beachfront                     142 non-null    float64
 11  breakfast                      142 non-null    float64
 12  building_staff                 142 non-null    flo

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_RATIO, random_state=42)

In [33]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)  # Replace with your model
])
pipeline.fit(, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  [4, 5, 6, 7, 33, 34, 35, 36,
                                                   39, 40, 41, 42, 43, 44, 45,
                                                   46, 47]),
                                                 ('cat', OneHotEncoder(),
                                                  [0, 1, 2, 38]),
                                                 ('passthrough', 'passthrough',
                                                  [8, 9, 10, 11, 12, 13, 14, 15,
                                                   16, 17, 18, 19, 20, 21, 22,
                                                   23, 24, 25, 26, 27, 28, 29,
                                                   30, 31, 32])])),
                ('model',
                 RandomForestRegressor(max_depth=64, n_estimators=58,
                                       random_state=123))])

In [35]:
y_pred = pipeline.predict(np.array(X))
val_mae = mean_absolute_error(y, y_pred)

In [36]:
val_mae

39643.474937153165

In [37]:
joblib.dump(pipeline, 'BasePriceModel.pkl')

['BasePriceModel.pkl']

In [39]:
sampleX = X.iloc[0]

array([377587.21588683])

In [46]:
type(sampleX)

numpy.ndarray

In [45]:
pipeline = joblib.load('/content/BasePriceModel.pkl')
sampleX = np.array(sampleX).reshape((1,48))
y_pred = pipeline.predict(sampleX)
print(y_pred)

[377587.21588683]
